# Probe Designer


## Environment


In [1]:
# basci env
import os
import sys
import pandas as pd
import time
import json
from tqdm import tqdm

# data process of file from ncbi
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import MeltingTemp as mt

# get gene data from ncbi
from Bio import Entrez

# blast and xml file process
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

# add package to sys var
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append("../lib")

# dir
workdir = './example_dataset'
os.makedirs(workdir, exist_ok=True)

current_time = time.localtime()
formatted_time = time.strftime("%Y%m%d_%H%M%S", current_time)

output = os.path.join(workdir, 'results', formatted_time)
pre_binding_dir = os.path.join(output, "pre_binding")
os.makedirs(output, exist_ok=True)

# basic variables
gene_name_list_tosearch = "gene_name_list_tosearch.txt"
pre_binding_file_suffix = "_pre_binding.fasta"
total_pre_binding_file_name = "_total.fasta"

# tmp file
gene_name_list_file = "immune.xlsx"
pre_binding_num_file = "pre_binding_num.json"
blast_results_file = "blast_results.xml"

In [2]:
organism = 'human'

## Get seq from ensembl dataset

In [ ]:
from lib.database_interaction import get_seqs_using_ensembl


gene_list = pd.read_excel(os.path.join(workdir, gene_name_list_file))["gene_name"].tolist()[:10]
sequences_of_all = dict()
for gene in gene_list:
    sequences = get_seqs_using_ensembl(gene=gene, species=organism, seq_type='cds')
    longest_item = max(sequences.items(), key=lambda x: len(x[1]))
    desc, sequence = longest_item
    sequences_of_all[desc] = sequence

## Binding site Searcher


In [12]:
from lib.search_binding import step_by_step, find_max_min_difference_fixed_length_subsequence, seq_minus

# Initiation of array
binding_site_FOIs = [
    "accession",
    "gene_name",
    "mol_type",
    "organism",
    "pos_on_seq",
    "binding",
    "Tm_l",
    "Tm_r",
    "wanted",
]
align_FOIs = ["align_num", "align_accession", "align_descrip", "plus/minus"]
FOI = pd.DataFrame(columns=binding_site_FOIs + align_FOIs)

# Search binding sites on mRNA sequence
file_out_dir = pre_binding_dir
try:
    os.mkdir(file_out_dir)
except:
    pass

pre_binding_num = {}

# initialization of file
with open(os.path.join(file_out_dir, total_pre_binding_file_name), "w") as handle:
    handle.write("")

for desc, seq in sequences_of_all.items():
    id, gene_name, mol_type = desc.split('|')
    minus_seq = seq_minus(seq)
    
    Tm_l, Tm_r, selected_substrings, pos_on_seq = step_by_step(
        minus_seq,
        BDS_len=40,
        BDS_num=50,
        min_gap=1,
        better_gap=40,
        gene=gene_name,
        G_min=0.25,
        G_max=0.7,
        G_consecutive=5,
        Tm_low=50,
        Tm_high=65,
    )
    
    record_list = []
    for i, pre_binding_tmp in enumerate(selected_substrings):
        record_list.append(
            SeqRecord(
                Seq(pre_binding_tmp),
                id="pre_binding" + str(i),
                description="|".join([id, gene_name, organism, mol_type]),
            )
        )

    # add information about binding sites to FOI
    add = pd.DataFrame(
        {
            "accession": [id] * len(selected_substrings),
            "gene_name": [gene_name] * len(selected_substrings),
            "mol_type": [mol_type] * len(selected_substrings),
            "organism": [organism] * len(selected_substrings),
            "binding": selected_substrings,
            "Tm_l": Tm_l,
            "Tm_r": Tm_r,
            "pos_on_seq": pos_on_seq,
        }
    )
    FOI = pd.concat([FOI, add], ignore_index=True)

    file_out = os.path.join(file_out_dir, gene_name + pre_binding_file_suffix)
    
    # write pre_binding to files
    with open(file_out, "w") as f:
        for new_record in record_list:
            SeqIO.write(new_record, f, "fasta")
    with open(file_out_dir + total_pre_binding_file_name, "a") as handle:
        for new_record in record_list:
            SeqIO.write(new_record, handle, "fasta")

    # record the num of pre_binding for each gene
    pre_binding_num[f"{id}_{gene_name}"] = len(selected_substrings)

with open(os.path.join(output, pre_binding_num_file), "w") as f:
    json.dump(pre_binding_num, f)

position_searching_CD3D-201: 100%|██████████| 374/374 [00:00<00:00, 40430.67it/s]


Gene CD3D-201: 	Not enough pos for 50 binding sites.
Gene CD3D-201: 	condition too harsh, loose to get better results
[0, 1, 2, 3, 5, 6, 7, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 37, 39, 40, 41, 42, 44, 45, 46, 47, 49, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 249, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 281, 282, 283, 284, 285, 286, 287]


position_searching_CD4-201: 100%|██████████| 1063/1063 [00:00<00:00, 23295.97it/s]


Gene CD4-201: 	condition too harsh, loose to get better results
[5, 23, 30, 37, 44, 54, 64, 73, 84, 91, 219, 226, 273, 308, 315, 322, 329, 336, 343, 350, 357, 364, 371, 381, 388, 395, 402, 409, 424, 497, 610, 661, 670, 677, 684, 691, 698, 705, 718, 762, 802, 809, 820, 827, 836, 958, 966, 975, 987, 994, 1014]


position_searching_CD8A-201: 100%|██████████| 528/528 [00:00<00:00, 10580.14it/s]


...

Gene AFP-201: 	Not enough pos for 50 binding sites.
Gene AFP-201: 	condition too harsh, loose to get better results
[35, 45, 133, 134, 135, 136, 152, 153, 154, 155, 156, 173, 174, 175, 176, 276, 277, 278, 279, 280, 281, 282, 292, 293, 294, 295, 296, 298, 299, 300, 301, 302, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 318, 319, 320, 321, 322, 323, 324, 325, 326, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 432, 433, 434, 435, 745, 746, 747, 879, 880, 881]


position_searching_GPC3-202: 100%|██████████| 1410/1410 [00:00<00:00, 55179.97it/s]


Gene GPC3-202: 	Not enough pos for 50 binding sites.
Gene GPC3-202: 	condition too harsh, loose to get better results
[114, 115, 116, 124, 125, 276, 277, 278, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 590, 591, 592, 593, 594, 806, 824, 825, 827, 828, 829, 830, 831, 833, 834, 835, 836, 842, 843, 858, 870, 1013, 1014, 1015, 1016, 1017, 1022, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1042, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1342, 1343, 1394, 1395, 1396, 1397, 1398, 1399, 1402]


position_searching_EPCAM-202: 100%|██████████| 785/785 [00:00<00:00, 87133.90it/s]


Gene EPCAM-202: 	Not enough pos for 50 binding sites.
Gene EPCAM-202: 	condition too harsh, loose to get better results
[408, 417, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 488, 489, 490, 491, 492, 493, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 510, 515, 516, 517, 518, 562, 563, 572, 573, 577, 578, 579, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 768, 769, 770, 771, 772, 773, 774]


## Blast and extract blast results

In [ ]:
# with open(file_out_dir + total_pre_binding_file_name, "r") as f:
#     fasta_string = f.read()
# txid = [2697049]  # organism

# # Submit BLAST search and get handle object
# handle = NCBIWWW.qblast(
#     program="blastn",
#     megablast="yes",
#     database="refseq_rna",
#     sequence=fasta_string,
#     url_base="https://blast.ncbi.nlm.nih.gov/Blast.cgi",
#     format_object="Alignment",
#     format_type="Xml",
# )

# # read handle object and save to a file
# with open(os.path.join(os.path.join(output, blast_results_file)), "w") as f:
#     f.write(handle.read())

In [19]:
# Extract interested information from blast_results
align_num = []

# read the id/plus-minus part/align_num
with open(os.path.join(output, blast_results_file), "r") as blast_output:
    blast_records = NCBIXML.parse(blast_output)
    loca = 0
    for blast_record in blast_records:
        align_accession = []
        align_descrip_list = []
        # get align num of each binding site
        length = len(blast_record.alignments)
        align_num.append(length)
        for i in range(length):
            descrip = blast_record.descriptions[i].title.split("|")
            # get accession and descrip of each align seq
            align_accession.append(descrip[3])
            align_descrip_list.append(descrip[-1])
        FOI.loc[loca, "align_accession"] = "|".join(str(_) for _ in align_accession)

        # add align_descrip to df
        FOI.loc[loca, "align_descrip"] = "|".join(str(_) for _ in align_descrip_list)

        # get plus/minus of each align seq
        p_m = [blast_record.alignments[_].hsps[0].frame[1] for _ in range(length)]

        # add plus/minus to df
        try:
            FOI.loc[loca, "plus/minus"] = ",".join([str(_) for _ in p_m])
        except:
            FOI.loc[loca, "plus/minus"] = "NAN"

        loca += 1

FOI["align_num"] = align_num

## Select wanted binding site


In [38]:
FOI["wanted"] = [True] * len(FOI)

In [39]:
# sieve for the suitable binding site
gene_name_list = [_.upper() for _ in gene_list]
gene_name_list_out = [i for i in gene_name_list]
for i in range(len(FOI)):
    # check gene_name
    gene_name = FOI.loc[i, "gene_name"]
    if gene_name.split('-')[0].upper() not in gene_name_list:
        FOI.loc[i, "wanted"] = False
    else:
        try:
            gene_name_list_out.remove(gene_name)
        except:
            pass

    # check DNA or mRNA type
    if FOI.loc[i, "wanted"] == True:
        if FOI.loc[i, "mol_type"] != "protein_coding":
            FOI.loc[i, "wanted"] = False
            print(FOI.loc[i, "mol_type"])

    # check gene_organism name
    if FOI.loc[i, "wanted"] == True:
        spe_ori, gene_ori = FOI.loc[i, "organism"], FOI.loc[i, "gene_name"].split('-')[0]
        descrip = FOI.loc[i, "align_descrip"].split("|")
        for des in descrip:
            if gene_ori not in des and spe_ori in des:
                FOI.loc[i, "wanted"] = False
                break

    # check plus/minus
    if FOI.loc[i, "wanted"] == True:
        if pd.isnull(FOI.loc[i, "plus/minus"]):
            FOI.loc[i, "wanted"] = False
        else:
            pm_list = FOI.loc[i, "plus/minus"].split(",")
            if "-1" not in pm_list:
                FOI.loc[i, "wanted"] = False

# write the whole information of interest to a excel file in tmp dir
FOI.to_excel(os.path.join(output, "probes_sieve.xlsx"))

out_tmp = FOI[FOI["wanted"] == True]
output_df = pd.DataFrame()
for gene in out_tmp.gene_name.unique():
    pos_of_True = list(out_tmp[out_tmp.gene_name == gene]["pos_on_seq"])
    best_pos = find_max_min_difference_fixed_length_subsequence(
        pos_of_True,
        length=3,
        min_gap=40,
        better_gap=80,
        gene=gene,
    )
    out_subset = out_tmp[out_tmp.gene_name == gene]
    out_subset = out_subset[out_subset["pos_on_seq"].isin(best_pos)]
    output_df = pd.concat([output_df, out_subset])

# write the output to a xlsx file
output_df.to_excel(os.path.join(output, "probes_wanted.xlsx"))

Gene CXCL13-201: 	Not enough pos for 3 binding sites.
Gene CXCL13-201: 	condition too harsh, loose to get better results
[251, 252, 253, 254, 255, 256]
Gene GZMA-201: 	Not enough pos for 3 binding sites.
Gene GZMA-201: 	condition too harsh, loose to get better results
[439, 440, 441, 442, 443, 446, 447, 508, 509, 510, 511, 512, 513, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 532, 533, 538, 555, 556]
Gene CLEC9A-201: 	condition too harsh, loose to get better results
[78, 150, 486]
Gene S100A8-202: 	Not enough pos for 3 binding sites.
Gene S100A8-202: 	condition too harsh, loose to get better results
[186, 187, 188]
